- [X] see sql5 file
- [X] variable match and selection
- [ ] define inclusion-exclusion criteria
- [ ] define target trial (cite hernan)

# All the features (Slide 11, page 14)

### Demographic:
- Gender *
- Age (IQR) *
- Ethnicity (Race: White, black, multiple or others) *
### Diseases:
- Other diseases (icd9 codes) *
- Surgery *
- Pneumonia *
- Infection
- Sepsis (diagnosis at admission) *
- Dementia *
- Alzheimer's *
- Depression *
- Anxiety *
- Coma (drug induced, miscellaneous, combination)
- Metabolic acidosis
- Respiratory failure
- ARDS (diagnosis at admission)
- Airway Protection (diagnosis at admission)
- Chronic Obstructive Pulmonary Disease, Asthma, or other pulmonary disorder (diagnosis at admission)
- Chronic heart failure, myocardial infarction, or arrhythmia (diagnosis at admission)
- Cirrhosis or Liver failure (diagnosis at admission)
- Other (diagnosis at admission)
### Drugs:
- List of drugs prescribed *
- Received APD treatment - no (%) (before admission, before admission and randomization)
- APD prescribed *
- Distinct APD categories *
### Admission events:
- Admission category (surgical, medical, trauma, neurology/neurosurgical)
- Length of stay *
- Mechanical ventilation ^*
- Mechanical ventilation hours ^ *
- Sedatives use (yes)
- Morphine use (mg/day)
- Urea concentration (mmol/L)
- Urgent admission (%)
- Hx of cognitive impairment
- Hx of ETOH use 
- MAP (mean arterial blood pressure) at the time of ICU admission
- Use of Corticosteroids
- BUN (blood urea nitrogen) at the of ICU admission
- Admitted to Surgical ICU - no (%)
- Received assisted ventilation before randomization - no (%) Invasive and Noninvasive
- Shock before randomization - no (%)
- Median no of days from ICU admission to randomization - (IQR)
### Evaluation:
- APACHE-II score (Acute Physiology & Chronic Health Evaluation) (at randomization)
 - APSIII in place of Apache score https://github.com/MIT-LCP/mimic-code/issues/653
- Median Short-form IQCODE score (IQR)
- Median Charlson Comorbidity Index Score (IQR)
- Median SOFA score at randomization
- Hyperactive Delirium at Randomization- no (%)
- Hypoactive Delirium at Randomization - no (%)
### Outcomes:
- Death in hospital *
- Death in one year *

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
data = pd.read_csv("delirium_data.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'delirium_data.csv'

In [ ]:
data.shape

In [ ]:
data.columns

# Deleted deathtime
- ADMISSIONS.DEATHTIME (https://mimic.mit.edu/docs/iii/tables/admissions/)
 - If applicable, DEATHTIME provides the time of in-hospital death for the patient. Note that DEATHTIME is only present if the patient died in-hospital, and is almost always the same as the patient’s DISCHTIME. However, there can be some discrepancies due to typographical errors.
 - Use to find death in hospital

- PATIENTS.DOD (https://mimic.mit.edu/docs/iii/tables/patients/)
 - DOD is the date of death for the given patient. DOD_HOSP is the date of death as recorded in the hospital database. DOD_SSN is the date of death from the social security database. Note that DOD merged together DOD_HOSP and DOD_SSN, giving priority to DOD_HOSP if both were recorded.
 - Real death time
 
- Just DOD works fine.

In [ ]:
data = data.drop('deathtime', 1)

data = data.rename(columns={
    "dist_cat_count": "drug_categories_distinct_count", 
    "gender": "sex", 
    "patient_deathtime": "deathtime", 
    "mechvent": "mechvent_count", 
    "had_surgery": "surgery", 
    "had_pneu": "pneumonia"})

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
data.sofa = data.sofa.fillna(-1)
data.apsiii = data.apsiii.fillna(-1)

In [ ]:
data.icustay_id = data.icustay_id.fillna(0)
data.surgery = data.surgery.fillna(2)
data = data.astype({"icustay_id": int, "sofa": int, "apsiii": int, "sex": 'category', "ethnicity": 'category', "surgery": int})

In [ ]:
data.info()

In [ ]:
data.sample(10)

# Checking for unique values and resolving if necessary

In [ ]:
print(len(data.subject_id.unique()))
print(len(data.hadm_id.unique()))
print(len(data.icustay_id.unique()))

In [ ]:
print("number of cases:", data[data.icustay_id.duplicated(keep=False)].shape)
data[data.icustay_id.duplicated(keep=False)]

Seems like they are six unique icu stay case, just with icustay_id missing, so we can ignore this issue now.

In [ ]:
print("number of cases:", data[data.hadm_id.duplicated(keep=False)].shape)
data[data.hadm_id.duplicated(keep=False)].head(5)

In [ ]:
temp = data[data.hadm_id.duplicated(keep=False)]
for i in temp.subject_id.unique():
    temp2 = temp[temp.subject_id == i]
    if (temp2.shape[0] != len(temp2.icustay_id.unique())): # check for unique icustay_id for each individual
        print(i)

They all have unique icustay_id, so treating as individual persons

In [ ]:
print("number of cases:", data[data.subject_id.duplicated(keep=False)].shape)
data[data.subject_id.duplicated(keep=False)].head(5)

# Explore other icd9 codes

In [ ]:
# data[data.icd9_codes.str.contains("518")]
all_icd_codes = pd.DataFrame(columns = ["shortcode", "longcode", "name", "count"])
for i in data.index:
    codes = data.icd9_codes[i].split("\n")
    for c in codes:
        shortcode = c[0:3]
        longcode = c.split(":")[0]
        name = c.split(": ")[1]
        ind_for_data_entry = all_icd_codes[all_icd_codes.longcode == longcode]
        if (len(ind_for_data_entry) == 0):
            all_icd_codes = pd.concat([all_icd_codes, pd.DataFrame(data={"shortcode": shortcode, "longcode":longcode, "name":name, "count":1}, index=[0])], axis=0, ignore_index=True)
        else:
            all_icd_codes.at[ind_for_data_entry.index[0],"count"] += 1

In [ ]:
all_icd_codes.shape

In [ ]:
# pd.set_option('display.max_colwidth', None)

- https://www.findacode.com/search/search.php
- http://www.icd9data.com/2015/Volume1/

- Other diseases (icd9 codes) *
- Surgery *
- Pneumonia *
- Infection
- Sepsis (diagnosis at admission) *
- Dementia *
- Alzheimer's *
- Depression *
- Anxiety *
- (COMPLEX) Coma (drug induced, miscellaneous, combination) [use name.str search with coma]
- Metabolic acidosis * [276.2]
- (COMPLEX) Respiratory failure [518.51, 518.53, 518.81, 581.83, 581.84, 799.1]
- (COMPLEX) ARDS (diagnosis at admission) [its complex since requires many other data, https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0257056]
- Airway Protection * (diagnosis at admission) [496] 
- [490-496] Chronic Obstructive Pulmonary Disease, [493] Asthma, or [496] other pulmonary disorder (diagnosis at admission)
 - COPD * [490-496]: a group of diseases that cause airflow blockage and breathing-related problems
- [428] Chronic heart failure, [410] myocardial infarction, or [427] AF and arrhythmia (diagnosis at admission)
 - heart issues *
- [571] Cirrhosis or [570] Liver failure (diagnosis at admission)
 - Liver issues *
- Other (diagnosis at admission)

In [ ]:
data[data.icd9_codes.str.contains("496: Chr airway obstruct NEC")].shape

In [ ]:
all_icd_codes[all_icd_codes.name.str.contains("sepsis")]

In [ ]:
data['met_acidosis'] = np.where(data.icd9_codes.str.contains("2764: Mixed acid-base bal dis"), 1, 0)
data['airway_obs'] = np.where(data.icd9_codes.str.contains("496: Chr airway obstruct NEC"), 1, 0)

copd_codes = ['490: Bronchitis NOS', '49121: Obs chr bronc w(ac) exac', 
              '4928: Emphysema NEC', '49392: Asthma NOS w (ac) exac', 
              '4941: Bronchiectasis w ac exac', '4957: "ventilation" pneumonit', 
              '496: Chr airway obstruct NEC']
data['copd'] = np.where(data.icd9_codes.str.contains('|'.join(copd_codes)), 1, 0)

liver_codes = ['570: Acute necrosis of liver', '5715: Cirrhosis of liver NOS']
data['liver_disease'] = np.where(data.icd9_codes.str.contains('|'.join(liver_codes)), 1, 0)

heart_codes = []
heart_icd_codes = all_icd_codes[all_icd_codes.shortcode.str.contains("410|427|428")]
for i in heart_icd_codes.index:
    heart_codes.append(heart_icd_codes.longcode[i] + ": " + heart_icd_codes.name[i])
print(heart_codes)
data['heart_disease'] = np.where(data.icd9_codes.str.contains('|'.join(heart_codes)), 1, 0)

In [ ]:
sorted_codes = all_icd_codes.sort_values('count', ascending=False)
sorted_codes.head(31)

# Formatting the datasets for use in exploratory, ml and causal analysis

In [ ]:
from collections import Counter

In [ ]:
def label_drug_name(row, frequency):
    drugs_list = row['drugs'].lower().split('\n')
    drugs_list_counter = Counter(drugs_list)
    if (frequency == 0):
        value, count = drugs_list_counter.most_common()[0]
    elif (frequency == 1):
        if (len(drugs_list_counter) > 1):
            value, count = drugs_list_counter.most_common()[1]
        else:
            value = 'NA'
    else:
        raise AttributeError("Frequency must be 0 or 1")
    return value

In [ ]:
def label_drug_type(row):
    if "no drug" in row['drug_1']:
        return 'ND'
    if "haloperidol" in row['drug_1']:
        return 'HL'
    if "ziprasidone" in row['drug_1']:
        return 'ZP'
    return 'OD'

In [ ]:
def label_timelength(row):
    if (pd.isna(row['drugstarttime'])):
        return -1
    
    if (pd.notna(row["deathtime"])):
        final_time = row["deathtime"]
    else:
        final_time = row["dischtime"]
            
    length = pd.Timedelta(pd.to_datetime(final_time) - pd.to_datetime(row["drugstarttime"])).total_seconds()
    length_in_days = round(length/ 86400.0) # in days
    return length_in_days

In [ ]:
def label_age(row):
    if row['age'] > 299:
        return 90
    else:
        return row['age']
    return row['age']

In [ ]:
# def label_sex(row):
#     if row['gender'] == 'F':
#         return 1
#     else:
#         return 0
#     return 1

In [ ]:
# https://grants.nih.gov/grants/guide/notice-files/not-od-15-089.html
def label_ethnicity(row):
    if (('WHITE' in row["ethnicity"].upper()) | ('EASTERN' in row["ethnicity"].upper())):
        return 1
    if ('BLACK' in row["ethnicity"].upper()):
        return 2
    if (('HISPANIC' in row["ethnicity"].upper()) | ('LATINO' in row["ethnicity"].upper())):
        return 3
    if ('ASIAN' in row["ethnicity"].upper()):
        return 4
    if (('AMERICAN' in row["ethnicity"].upper()) | ('ALASKA' in row["ethnicity"].upper())):
        return 5
    return 0

In [ ]:
def label_time_to_mechvent(row):
    if ((isinstance(row["mechvent_duration_hours"], int)) | ((isinstance(row["mechvent_duration_hours"], float)))):
        return float(row["mechvent_duration_hours"])
    else:
        if ("," in row["mechvent_duration_hours"]):
            return float(max(row["mechvent_duration_hours"].split(",")))
        else:
            return float(row["mechvent_duration_hours"])
    return 0

In [ ]:
import datetime

def label_time_to_mechvent(row):
    if pd.isna(row['mechvent_starttime']):
        return -1.0
    else:
        datetimeFormat = '%Y-%m-%d %H:%M:%S'
        date1 = row['mechvent_starttime']
        date2 = row['admittime']
        diff = datetime.datetime.strptime(date1, datetimeFormat) - datetime.datetime.strptime(date2, datetimeFormat)
        return (diff.total_seconds() / 86400)
    return -1.0

In [ ]:
def label_mechvent(row):
    if row['mechvent_count'] == 0:
        return 0
    else:
        return 1
    return 0

In [ ]:
def label_mechvent_hours(row):
    if ((isinstance(row["mechvent_duration_hours"], int)) | ((isinstance(row["mechvent_duration_hours"], float)))):
        return float(row["mechvent_duration_hours"])
    else:
        if ("," in row["mechvent_duration_hours"]):
            return float(max(row["mechvent_duration_hours"].split(",")))
        else:
            return float(row["mechvent_duration_hours"])
    return 0

In [ ]:
def label_death_type(row):
    if (pd.isna(row['deathtime']) or pd.isna(row['deathtime'])):
        return -1
        
    datetimeFormat = '%Y-%m-%d %H:%M:%S'
    date1 = row['deathtime']
    date2 = row['dischtime']
    diff = datetime.datetime.strptime(date1, datetimeFormat) - datetime.datetime.strptime(date2, datetimeFormat)
    days = (diff.total_seconds() / 86400)
    if (days <= 30):
        return 1
    elif ((days > 30) & (days <= 90)):
        return 2
    elif ((days > 90) & (days <= 366)):
        return 3
    else:
        return 4
    return 0

In [ ]:
data['drug_1'] = data.apply(lambda row: label_drug_name(row, 0), axis=1)
data['drug_2'] = data.apply(lambda row: label_drug_name(row, 1), axis=1)
data['drug_group'] = data.apply(lambda row: label_drug_type(row), axis=1)
data['drug_timelength'] = data.apply(lambda row: label_timelength(row), axis=1)
data['age'] = data.apply(lambda row: label_age(row), axis=1)
data['race'] = data.apply(lambda row: label_ethnicity(row), axis=1)
data['time_to_mechvent'] = data.apply(lambda row: label_time_to_mechvent(row), axis=1)
data['mechvent'] = data.apply(lambda row: label_mechvent(row), axis=1)
data['time_in_mechvent'] = data.apply(lambda row: label_mechvent_hours(row), axis=1)
data['death_timeline'] = data.apply(lambda row: label_death_type(row), axis=1)

In [ ]:
data.shape

In [ ]:
data.sample(3)

In [ ]:
data.columns

In [ ]:
# dropping: 
# 'ethnicity', 'mechvent_starttime','drugstarttime', 'admittime', 'dischtime', 
# 'deathtime', 'mechvent_duration_hours', 'drugs', 'drug_categories', 'death_one_year', 'drug_1', 'drug_2'
data = data[['subject_id', 'hadm_id', 'icustay_id', 'sex', 'age', 'race', 'icd9_codes',
             'sofa', 'apsiii', 'surgery', 'pneumonia', 'sepsis', 'dementia', 'alzheimers', 'depression', 
             'anxiety', 'met_acidosis', 'airway_obs', 'copd', 'liver_disease', 'heart_disease', 
             'mechvent', 'mechvent_count', 'drug_group', 'drug_categories_distinct_count', 'drug_timelength', 
             'death_hosp', 'death_timeline', 'los_days', 'time_to_mechvent', 'time_in_mechvent']]

In [ ]:
data = data.astype({"drug_group": 'category'})

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.sample(5)

In [ ]:
# count the na s and fix it
data.isna().sum()

In [ ]:
data.los_days = data.los_days.fillna(-1)
data.time_in_mechvent = data.time_in_mechvent.fillna(-1)
data.isna().sum()

In [ ]:
data.to_csv('delirium_data_cleaned.csv', header=True, index=False)
sorted_codes.to_csv('delirium_data_icd_codes.csv', header=True, index=False)